In [13]:
import sys
import os

# 1. Agregamos la carpeta superior al sistema (para encontrar config.py)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# 2. Ahora sí importamos tu configuración
try:
    from config import PATHS
    print("✅ Configuración importada correctamente.")
except ImportError:
    print("❌ No se encontró el archivo 'config.py' en la carpeta superior.")
    # Definimos PATHS dummy para que no rompa si falla el import
    PATHS = {"raw_sla": "."} 

# 3. Resto de librerías
import glob
import re
import datetime
import pandas as pd
import numpy as np

✅ Configuración importada correctamente.


In [14]:

def obtener_rango_fechas(nombre_archivo):
    """
    Extrae fechas usando TU patrón validado: 'd-m-y al d-m-y'
    Retorna: Fecha Inicio, Fecha Fin, Etiqueta (Ej: 'ENE 2026 Q1')
    """
    # 1. Limpieza básica
    nombre_limpio = os.path.basename(nombre_archivo).lower()
    
    # 2. TU REGEX (La que sí funcionaba)
    # Busca algo como: "1-12-2025 al 15-12-2025"
    patron = r"(\d{1,2}-\d{1,2}-\d{4})\s+al\s+(\d{1,2}-\d{1,2}-\d{4})"
    
    match = re.search(patron, nombre_limpio)
    
    if not match:
        return None, None, None

    try:
        str_inicio, str_fin = match.groups()
        
        # 3. Conversión directa (Asumiendo formato dd-mm-yyyy del nombre de archivo)
        f_inicio = datetime.datetime.strptime(str_inicio, "%d-%m-%Y")
        f_fin = datetime.datetime.strptime(str_fin, "%d-%m-%Y")

        # 4. Lógica de Quincena (Q1 vs Q2)
        # Regla: Si la fecha fin es el día 15 (o antes), es la primera quincena.
        #        Si la fecha fin es mayor al 15 (ej. 30 o 31), es la segunda quincena.
        quincena_str = "Q1" if f_fin.day <= 15 else "Q2"
        
        # 5. Nombre del Mes (En Español)
        # Usamos f_fin.month para saber a qué mes pertenece el cierre
        meses = ["", "ENE", "FEB", "MAR", "ABR", "MAY", "JUN", 
                 "JUL", "AGO", "SEP", "OCT", "NOV", "DIC"]
        
        # Construimos la etiqueta: "DIC 2025 Q1"
        nombre_etiqueta = f"{meses[f_fin.month]} {f_fin.year} {quincena_str}"
        
        return f_inicio, f_fin, nombre_etiqueta

    except Exception as e:
        print(f"[red]❌ Error calculando fechas en {nombre_archivo}: {e}[/]")
        return None, None, None

In [15]:
import os
import glob
import re
import datetime
import pandas as pd
import numpy as np

# ==========================================
# 1. FUNCIONES UTILITARIAS
# ==========================================
def obtener_rango_fechas(nombre_archivo):
    try:
        nombre_limpio = os.path.basename(nombre_archivo).lower()
        patron = r"(\d{1,2}-\d{1,2}-\d{4})\s+al\s+(\d{1,2}-\d{1,2}-\d{4})"
        match = re.search(patron, nombre_limpio)
        if not match: return None, None, None

        str_inicio, str_fin = match.groups()
        f_inicio = datetime.datetime.strptime(str_inicio, "%d-%m-%Y")
        f_fin = datetime.datetime.strptime(str_fin, "%d-%m-%Y")

        quincena_str = "Q1" if f_fin.day <= 15 else "Q2"
        meses = ["", "ENE", "FEB", "MAR", "ABR", "MAY", "JUN", 
                 "JUL", "AGO", "SEP", "OCT", "NOV", "DIC"]
        nombre_etiqueta = f"{meses[f_fin.month]} {f_fin.year} {quincena_str}"
        return f_inicio, f_fin, nombre_etiqueta
    except Exception:
        return None, None, None

def limpiar_fechas_mixtas(series):
    """
    Función maestra para columnas que vienen sucias (mezcla de fechas, números y texto).
    """
    # 1. Si ya es fecha (datetime), no tocamos nada
    if pd.api.types.is_datetime64_any_dtype(series):
        return series

    # 2. Convertir a numérico (para seriales Excel)
    # Los seriales válidos están entre 35000 (año 1995) y 60000 (año 2064)
    series_nums = pd.to_numeric(series, errors='coerce')
    mask_es_serial = (series_nums > 35000) & (series_nums < 60000)
    
    # Convertimos los seriales seguros
    fechas_excel = pd.to_datetime(series_nums[mask_es_serial], unit='D', origin='1899-12-30')
    
    # 3. El resto lo tratamos como texto con dayfirst=True
    # Solo procesamos lo que NO era serial
    resto = series[~mask_es_serial].astype(str).str.strip()
    # errors='coerce' pondrá NaT a la basura (teléfonos, textos raros)
    fechas_texto = pd.to_datetime(resto, dayfirst=True, errors='coerce')
    
    # 4. Combinar: Preferencia a los seriales, relleno con texto
    return fechas_excel.combine_first(fechas_texto)

# ==========================================
# 2. CONFIGURACIÓN
# ==========================================

# Asegúrate de que PATHS esté definido
# PATHS = {"raw_sla": "C:/Ruta/..."}

NOC_USERS = [
    "GFARFAN", "JVELASQUEZ", "JOLUGO", "KUSEA", "SLOPEZ",
    "EDESPINOZA", "SANDYJIM", "JOCASTILLO", "JESUSGARCIA",
    "DFUENTES", "JOCANTO", "IXMONTILLA", "OMTRUJILLO",
    "LLZERPA", "LUIJIMENEZ"
]

EXCLUIR_SOLUCIONES = [
    "CAMBIO DE CLAVE", "CLIENTE SOLICITO REEMBOLSO", 
    "LLAMADAS DE AGENDAMIENTO", "ORDEN REPETIDA"
]

COLS_EXTRACCION = [
    "FechaInicio", "FechaFin", "FechaInicioQuincena", "Quincena Evaluada",
    "N° Contrato", "Estatus contrato", "N° Orden", "Estatus_orden",
    "Fecha Creacion", "Fecha Impresion", "Fecha Finalizacion", 
    "Grupo Afinidad", "Detalle Orden", "Franquicia",
    "Grupo Trabajo", "Usuario Emisión", "Usuario Impresión", 
    "Usuario Final", "Solucion Aplicada"
]

ORDEN_FINAL = [
    "Fecha Creacion", "Fecha Impresion", "Fecha Finalizacion",
    "SLA Resolucion", "SLA Despacho", "SLA Impresion",
    "Clasificacion", "N° Orden", "N° Contrato", 
    "Grupo Trabajo", "Usuario Final", "Solucion Aplicada",
    "Estatus_orden", "Detalle Orden", "Grupo Afinidad", 
    "Franquicia", "Usuario Emisión", "Usuario Impresión", "Estatus contrato",
    "Quincena Evaluada", "FechaInicio", "FechaFin"
]

ruta_origen = PATHS.get("raw_sla")
if not ruta_origen or not os.path.exists(ruta_origen):
    print(f"❌ Error: Ruta inválida {ruta_origen}")
else:
    archivos = glob.glob(os.path.join(ruta_origen, "*.xlsx"))
    print(f"📂 Se encontraron {len(archivos)} archivos.")

    dataframes_procesados = []

    # ==========================================
    # 3. PROCESAMIENTO
    # ==========================================

    for archivo in archivos:
        nombre_archivo = os.path.basename(archivo)

        if nombre_archivo.startswith("~$") or "Consolidado" in nombre_archivo:
            continue
        
        fecha_inicio, fecha_fin, quincena_nombre = obtener_rango_fechas(nombre_archivo)
        if not fecha_inicio:
            print(f"⚠️ Saltando {nombre_archivo} (Sin fechas válidas)")
            continue

        try:
            # --- C. LECTURA (CAMBIO IMPORTANTE: Sin dtype=str) ---
            # Al quitar dtype=str, dejamos que 'calamine' detecte las fechas nativas de Excel.
            # Esto evita la ambigüedad 1-11 (Enero vs Noviembre).
            df = pd.read_excel(archivo, engine="calamine")
            
            if df.empty: continue

            # --- D. CORRECCIÓN DE FECHAS INTELIGENTE ---
            cols_fecha = ["Fecha Creacion", "Fecha Emisión", "Fecha Final", 
                          "Fecha Impresion", "Fecha Cierre", "Fecha Finalizacion"]
            
            for col in cols_fecha:
                if col in df.columns:
                    # Usamos la función que respeta lo que ya es fecha y arregla lo demás
                    df[col] = limpiar_fechas_mixtas(df[col])

            # --- E. FILTRO CRUZADO ---
            if "Fecha Finalizacion" not in df.columns:
                print(f"❌ {nombre_archivo} falta Fecha Finalizacion")
                continue

            # Filtramos solo por fecha finalización
            df_filtrado = df.query("@fecha_inicio <= `Fecha Finalizacion` <= @fecha_fin").copy()
            
            if len(df_filtrado) == 0:
                print(f"⚠️ {nombre_archivo}: 0 filas. (Rango: {fecha_inicio.date()} al {fecha_fin.date()})")
                continue

            # --- F. ENRIQUECIMIENTO ---
            df_filtrado = df_filtrado.assign(
                FechaInicio = fecha_inicio,
                FechaFin = fecha_fin,
                FechaInicioQuincena = fecha_inicio,
                Quincena_Evaluada = quincena_nombre
            ).rename(columns={"Quincena_Evaluada": "Quincena Evaluada"})

            # --- G. LIMPIEZA NEGOCIO ---
            df_filtrado = df_filtrado.assign(
                Solucion_Norm = lambda x: x["Solucion Aplicada"].fillna("").astype(str).str.upper(),
                Grupo_Norm    = lambda x: x["Grupo Trabajo"].fillna("").astype(str).str.upper(),
                Detalle_Norm  = lambda x: x["Detalle Orden"].fillna("").astype(str).str.upper(),
                Estatus_Norm  = lambda x: x["Estatus_orden"].fillna("").astype(str).str.upper()
            )
            
            mask_excluir = (
                df_filtrado["Solucion_Norm"].isin(EXCLUIR_SOLUCIONES) |
                df_filtrado["Grupo_Norm"].str.contains("GT API FIBEX", na=False) |
                (df_filtrado["Detalle_Norm"] == "PRUEBA DE INTERNET") |
                df_filtrado["Estatus_Norm"].str.contains("CREACIÓN", na=False)
            )
            
            df_final = df_filtrado[~mask_excluir].reindex(columns=COLS_EXTRACCION)
            dataframes_procesados.append(df_final)
            print(f" ✅ {nombre_archivo} -> {len(df_final)} filas OK")

        except Exception as e:
            print(f" ❌ Error crítico en {nombre_archivo}: {e}")

    # ==========================================
    # 4. CONSOLIDACIÓN
    # ==========================================

    print(f"\nConsolidando {len(dataframes_procesados)} DataFrames...")

    if dataframes_procesados:
        review_r = (
            pd.concat(dataframes_procesados, ignore_index=True)
            .assign(
                Grupo_Norm   = lambda x: x['Grupo Trabajo'].fillna('').astype(str).str.upper(),
                Usuario_Norm = lambda x: x['Usuario Final'].fillna('').astype(str).str.upper()
            )
            .assign(
                Clasificacion = lambda x: np.select(
                    condlist=[
                        x['Usuario_Norm'].isin(NOC_USERS),
                        x['Grupo_Norm'].isin(NOC_USERS),
                        x['Usuario_Norm'].str.contains('NOC', na=False),
                        x['Grupo_Norm'].str.contains('OPERACIONES', na=False)
                    ],
                    choicelist=['NOC', 'NOC', 'NOC', 'OPERACIONES'],
                    default='MESA DE CONTROL'
                )
            )
            .assign(
                SLA_Despacho   = lambda x: x['Fecha Finalizacion'] - x['Fecha Impresion'],
                SLA_Resolucion = lambda x: x['Fecha Finalizacion'] - x['Fecha Creacion'],
                SLA_Impresion  = lambda x: x['Fecha Impresion'] - x['Fecha Creacion']
            )
            .query("SLA_Resolucion.notna()")
            .drop(columns=['Grupo_Norm', 'Usuario_Norm'])
            .rename(columns={
                "SLA_Resolucion": "SLA Resolucion",
                "SLA_Despacho":   "SLA Despacho",
                "SLA_Impresion":  "SLA Impresion"
            })
            .reindex(columns=ORDEN_FINAL)
        )

        print(f"📊 Resultado Final: {len(review_r)} filas consolidadas.")
        display(review_r.head())
        
    else:
        print("⚠️ No se generaron datos.")

📂 Se encontraron 13 archivos.
 ✅ Data - SLA 1-10-2025 al 15-11-2025.xlsx -> 29807 filas OK
 ✅ Data - SLA 1-11-2025 al 15-12-2025.xlsx -> 27646 filas OK
 ✅ Data - SLA 1-12-2025 al 15-1-2026.xlsx -> 28404 filas OK
 ✅ Data - SLA 1-7-2025 al 15-8-2025.xlsx -> 31707 filas OK
 ✅ Data - SLA 1-8-2025 al 15-9-2025.xlsx -> 29699 filas OK
 ✅ Data - SLA 1-9-2025 al 15-10-2025.xlsx -> 29250 filas OK
 ✅ Data - SLA 15-10-2025 al 30-11-2025.xlsx -> 29265 filas OK
 ✅ Data - SLA 15-11-2025 al 31-12-2025.xlsx -> 27892 filas OK
 ✅ Data - SLA 15-7-2025 al 31-8-2025.xlsx -> 31385 filas OK
 ✅ Data - SLA 15-8-2025 al 30-9-2025.xlsx -> 30616 filas OK
 ✅ Data - SLA 15-9-2025 al 31-10-2025.xlsx -> 30805 filas OK
⚠️ Saltando Data - SLA Agosto 2025.xlsx (Sin fechas válidas)

Consolidando 11 DataFrames...
📊 Resultado Final: 326476 filas consolidadas.


,Fecha Creacion,Fecha Impresion,Fecha Finalizacion,SLA Resolucion,SLA Despacho,SLA Impresion,Clasificacion,N° Orden,N° Contrato,Grupo Trabajo,...,Estatus_orden,Detalle Orden,Grupo Afinidad,Franquicia,Usuario Emisión,Usuario Impresión,Estatus contrato,Quincena Evaluada,FechaInicio,FechaFin
0,2025-10-08 08:22:00,2025-10-08 09:03:00,2025-10-08 09:03:00,0 days 00:41:00,0 days 00:00:00,0 days 00:41:00,NOC,V200444,H9131,GERLY FARFAN,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,AIHPXIX,GFARFAN,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
1,2025-11-04 17:40:00,2025-11-04 17:47:00,2025-11-04 17:56:00,0 days 00:16:00,0 days 00:09:00,0 days 00:07:00,MESA DE CONTROL,V15834,H9162,JHOAN ANDRES YAMARTE (NOC),...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,JYAMARTE,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
2,2025-11-02 18:01:00,2025-11-02 18:36:00,2025-11-02 18:36:00,0 days 00:35:00,0 days 00:00:00,0 days 00:35:00,MESA DE CONTROL,V14254,H9162,CARLOS GONZALEZ (NOC),...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,CARLGONZALEZ,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
3,2025-10-12 18:10:00,2025-10-12 18:31:00,2025-10-12 18:31:00,0 days 00:21:00,0 days 00:00:00,0 days 00:21:00,MESA DE CONTROL,V00803,H9162,DIEGO REYES,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,DIREYES,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
4,2025-10-07 21:00:00,2025-10-07 23:00:00,2025-10-07 23:06:00,0 days 02:06:00,0 days 00:06:00,0 days 02:00:00,NOC,V200371,H9162,DENNY FUENTE,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,DFUENTES,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15


In [16]:
review_r = review_r[review_r['Fecha Creacion'] >= review_r['FechaInicio']]
dec = review_r[review_r.FechaInicio == pd.Timestamp('2025-11-01')]

display(dec)

,Fecha Creacion,Fecha Impresion,Fecha Finalizacion,SLA Resolucion,SLA Despacho,SLA Impresion,Clasificacion,N° Orden,N° Contrato,Grupo Trabajo,...,Estatus_orden,Detalle Orden,Grupo Afinidad,Franquicia,Usuario Emisión,Usuario Impresión,Estatus contrato,Quincena Evaluada,FechaInicio,FechaFin
29807,2025-11-01 05:30:00,2025-11-01 05:39:00,2025-11-01 05:41:00,0 days 00:11:00,0 days 00:02:00,0 days 00:09:00,MESA DE CONTROL,V13367,V110203,LUIS JOAQUIN VILLEGAS,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,LJVILLEGAS,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29808,2025-11-01 07:10:00,2025-11-01 07:21:00,2025-11-03 10:49:00,2 days 03:39:00,2 days 03:28:00,0 days 00:11:00,OPERACIONES,A07218,A6827,OPERACIONES ARAGUA,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX ARAGUA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29809,2025-11-01 07:17:00,2025-11-01 07:32:00,2025-11-03 13:48:00,2 days 06:31:00,2 days 06:16:00,0 days 00:15:00,MESA DE CONTROL,V13380,V25679,SOLUCIONES TECNO TYC 2025,...,FINALIZADA,FALLA LOS,HOGAR,FIBEX VALENCIA,APICONEXVE,KUSEA,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29810,2025-11-01 07:42:00,2025-11-01 07:43:00,2025-11-01 07:43:00,0 days 00:01:00,0 days 00:00:00,0 days 00:01:00,MESA DE CONTROL,V13389,V68853,CARLOS GONZALEZ (NOC),...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,CARLGONZALEZ,CARLGONZALEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
29811,2025-11-01 07:00:00,2025-11-01 07:35:00,2025-11-01 07:43:00,0 days 00:43:00,0 days 00:08:00,0 days 00:35:00,MESA DE CONTROL,V13372,V110203,WILLCLAR MARTINEZ,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57448,2025-12-13 08:00:00,2025-12-13 08:48:00,2025-12-13 08:48:00,0 days 00:48:00,0 days 00:00:00,0 days 00:48:00,NOC,V334819,V113172,SANDY JOSE JIMENEZ BETANCOURT,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,SANDYJIM,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
57449,2025-12-13 08:00:00,2025-12-13 08:47:00,2025-12-13 08:52:00,0 days 00:52:00,0 days 00:05:00,0 days 00:47:00,MESA DE CONTROL,MGTA180457,MGTA6616,WILLCLAR MARTINEZ,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX MARGARITA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
57450,2025-12-13 08:00:00,2025-12-13 08:53:00,2025-12-13 08:59:00,0 days 00:59:00,0 days 00:06:00,0 days 00:53:00,MESA DE CONTROL,A67841,A59583,WILLCLAR MARTINEZ,...,FINALIZADA,CAIDAS SEGUIDAS DE INTERNET,HOGAR,FIBEX ARAGUA,APICONEXVE,WILLMARTINEZ,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15
57451,2025-12-13 08:03:00,2025-12-13 08:59:00,2025-12-13 08:59:00,0 days 00:56:00,0 days 00:00:00,0 days 00:56:00,NOC,SC182892,SC12660,SANDY JOSE JIMENEZ BETANCOURT,...,FINALIZADA,CAIDAS SEGUIDAS DE INTERNET,HOGAR,FIBEX SAN CARLOS,MLUCENA,SANDYJIM,ACTIVO,DIC 2025 Q1,2025-11-01,2025-12-15


In [26]:
import pandas as pd
import numpy as np

ruta=r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\SLA_Gold.parquet"
sla = pd.read_parquet(ruta)

# Asegurar que la columna de fecha sea datetime (evita comparaciones con strings)
sla["Fecha Finalizacion"] = pd.to_datetime(sla["Fecha Finalizacion"], errors="coerce")

decem = sla[sla["Fecha Finalizacion"] >= pd.Timestamp('2026-01-01')]

# Calcular medias solo para columnas numéricas y de tipo timedeltas (SLA)
num_mean = decem.select_dtypes(include=[np.number]).mean()
td_mean = decem.select_dtypes(include=['timedelta64[ns]']).mean()
mean_sla = pd.concat([num_mean, td_mean])

print("Mean SLA:")
print(mean_sla)

Mean SLA:
SLA Resolucion   0 days 18:05:58.782150899
SLA Despacho     0 days 15:58:04.275792077
SLA Impresion    0 days 02:07:54.506358821
SLA Resolucion   0 days 18:05:58.782150899
SLA Despacho     0 days 15:58:04.275792077
SLA Impresion    0 days 02:07:54.506358821
dtype: timedelta64[ns]


In [32]:
import pandas as pd

ruta = r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\SLA_Gold.parquet"
sla = pd.read_parquet(ruta)
display(sla)





,Fecha Creacion,Fecha Impresion,Fecha Finalizacion,SLA Resolucion Min,SLA Despacho Min,SLA Impresion Min,SLA Detalle Texto,Clasificacion,N° Orden,N° Contrato,...,Estatus_orden,Detalle Orden,Grupo Afinidad,Franquicia,Usuario Emisión,Usuario Impresión,Estatus contrato,Quincena Evaluada,FechaInicio,FechaFin
0,2025-10-08 08:22:00,2025-10-08 09:03:00,2025-10-08 09:03:00,41.0,0.0,41.0,0 days 00:41:00,NOC,V200444,H9131,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,AIHPXIX,GFARFAN,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
1,2025-11-04 17:40:00,2025-11-04 17:47:00,2025-11-04 17:56:00,16.0,9.0,7.0,0 days 00:16:00,MESA DE CONTROL,V15834,H9162,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,JYAMARTE,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
2,2025-11-02 18:01:00,2025-11-02 18:36:00,2025-11-02 18:36:00,35.0,0.0,35.0,0 days 00:35:00,MESA DE CONTROL,V14254,H9162,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,CARLGONZALEZ,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
3,2025-10-12 18:10:00,2025-10-12 18:31:00,2025-10-12 18:31:00,21.0,0.0,21.0,0 days 00:21:00,MESA DE CONTROL,V00803,H9162,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,DIREYES,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
4,2025-10-07 21:00:00,2025-10-07 23:00:00,2025-10-07 23:06:00,126.0,6.0,120.0,0 days 02:06:00,NOC,V200371,H9162,...,FINALIZADA,SIN INTERNET,HOGAR,FIBEX VALENCIA,APICONEXVE,DFUENTES,ACTIVO,NOV 2025 Q1,2025-10-01,2025-11-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359319,2025-10-27 11:33:00,2025-10-27 14:15:00,2025-10-27 14:16:00,163.0,1.0,162.0,0 days 02:43:00,MESA DE CONTROL,MI0781,MI4742,...,FINALIZADA,USUARIO CLAVE ONT,HOGAR,FIBEX MIRANDA,AIHPXI,VGAMEZ,ACTIVO,OCT 2025 Q2,2025-09-15,2025-10-31
359320,2025-10-22 10:09:00,2025-10-22 12:26:00,2025-10-24 16:09:00,3240.0,3103.0,137.0,2 days 06:00:00,MESA DE CONTROL,C03223,C34323,...,FINALIZADA,FALLA LOS,HOGAR,FIBEX CARACAS,JOSHERNANDEZ,DOURODRIGUEZ,ACTIVO,OCT 2025 Q2,2025-09-15,2025-10-31
359321,2025-10-24 08:24:00,2025-10-24 08:40:00,2025-10-24 11:00:00,156.0,140.0,16.0,0 days 02:36:00,MESA DE CONTROL,GU2028,GU29800,...,FINALIZADA,FALLA LOS,HOGAR,FIBEX GUARICO,RAREYES,DFUENTES,ACTIVO,OCT 2025 Q2,2025-09-15,2025-10-31
359322,2025-09-29 13:27:00,2025-09-29 14:47:00,2025-10-01 12:20:00,2813.0,2733.0,80.0,1 days 22:53:00,MESA DE CONTROL,LCH29376,LCH43879,...,FINALIZADA,FALLA LOS,HOGAR,FIBEX ANZOATEGUI,EGUACARAN,ANTHGONZALEZ,ACTIVO,OCT 2025 Q2,2025-09-15,2025-10-31


In [30]:
import pandas as pd

ruta = r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\IdF_Gold.parquet"
idf = pd.read_parquet(ruta)
display(idf)




,Quincena Evaluada,FechaInicio,FechaFin,N° Contrato,Estatus contrato,N° Orden,Estatus_orden,Fecha Creacion,Fecha Impresion,Fecha Finalizacion,Grupo Afinidad,Detalle Orden,Franquicia,Grupo Trabajo,Usuario Final,Solucion Aplicada,Clasificacion
0,NOV 2025 Q1,2025-10-01,2025-11-15,H9131,ACTIVO,V200444,FINALIZADA,2025-10-08 08:22:00,2025-10-08 09:03:00,2025-10-08 09:03:00,HOGAR,SIN INTERNET,FIBEX VALENCIA,GERLY FARFAN,GFARFAN,PROBLEMA DE WIFI,NOC
1,NOV 2025 Q1,2025-10-01,2025-11-15,H9162,ACTIVO,V15834,FINALIZADA,2025-11-04 17:40:00,2025-11-04 17:47:00,2025-11-04 17:56:00,HOGAR,SIN INTERNET,FIBEX VALENCIA,JHOAN ANDRES YAMARTE (NOC),JYAMARTE,CONFIGURACION DE EQUIPO,MESA DE CONTROL
2,NOV 2025 Q1,2025-10-01,2025-11-15,H9162,ACTIVO,V14254,FINALIZADA,2025-11-02 18:01:00,2025-11-02 18:36:00,2025-11-02 18:36:00,HOGAR,SIN INTERNET,FIBEX VALENCIA,CARLOS GONZALEZ (NOC),CARLGONZALEZ,FALLA GENERAL,MESA DE CONTROL
3,NOV 2025 Q1,2025-10-01,2025-11-15,H9162,ACTIVO,V00803,FINALIZADA,2025-10-12 18:10:00,2025-10-12 18:31:00,2025-10-12 18:31:00,HOGAR,SIN INTERNET,FIBEX VALENCIA,DIEGO REYES,DIREYES,PROBLEMA DE WIFI,MESA DE CONTROL
4,NOV 2025 Q1,2025-10-01,2025-11-15,H9162,ACTIVO,V200371,FINALIZADA,2025-10-07 21:00:00,2025-10-07 23:00:00,2025-10-07 23:06:00,HOGAR,SIN INTERNET,FIBEX VALENCIA,DENNY FUENTE,DFUENTES,CAMBIO DE VLAN,NOC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359319,OCT 2025 Q2,2025-09-15,2025-10-31,MI4742,ACTIVO,MI0781,FINALIZADA,2025-10-27 11:33:00,2025-10-27 14:15:00,2025-10-27 14:16:00,HOGAR,USUARIO CLAVE ONT,FIBEX MIRANDA,VICTOR GAMEZ,VGAMEZ,PROBLEMA DE WIFI,MESA DE CONTROL
359320,OCT 2025 Q2,2025-09-15,2025-10-31,C34323,ACTIVO,C03223,FINALIZADA,2025-10-22 10:09:00,2025-10-22 12:26:00,2025-10-24 16:09:00,HOGAR,FALLA LOS,FIBEX CARACAS,JOSE LUIS OLIVIO SANCHEZ,CLTORRES,CONECTOR DAñADO,MESA DE CONTROL
359321,OCT 2025 Q2,2025-09-15,2025-10-31,GU29800,ACTIVO,GU2028,FINALIZADA,2025-10-24 08:24:00,2025-10-24 08:40:00,2025-10-24 11:00:00,HOGAR,FALLA LOS,FIBEX GUARICO,SVH TECNOLOGIA MERCEDEES DEL L,LGESTRADA,CONECTOR DAñADO,MESA DE CONTROL
359322,OCT 2025 Q2,2025-09-15,2025-10-31,LCH43879,ACTIVO,LCH29376,FINALIZADA,2025-09-29 13:27:00,2025-09-29 14:47:00,2025-10-01 12:20:00,HOGAR,FALLA LOS,FIBEX ANZOATEGUI,WILFREDO MORALES,GBARBERA,CONECTOR DAñADO,MESA DE CONTROL


In [33]:
import pandas as pd
import os

df = pd.read_parquet(r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Atencion_Cliente_Gold.parquet")
display(df)


,N° Abonado,Documento,Cliente,Estatus,Fecha,Hora,Tipo Respuesta,Detalle Respuesta,Vendedor,Suscripción,Grupo Afinidad,Nombre Franquicia,Ciudad,Tipo de afluencia,Mes
0,A52976,5430449,ZOILA MARIA LANDAETA MARRERO,ACTIVO,2025-07-04,15:16:00,INFORMACION SERVICIO,FACTURACION,YUSMARY CEGARRA OFIC PROPATRIA CARACAS,30.0,HOGAR,FIBEX ARAGUA,TOVAR (COLONIA TOVAR),ATENCIÓN AL CLIENTE,Julio
1,A61422,21639385,YURANIS VICTORIA HERRERA,ACTIVO,2025-07-04,09:51:00,INFORMACION SERVICIO,PLANES Y TARIFAS,JESUS BRICEÑO ZAVALA OFIC PASEO LAS INDUSTRIAS,30.0,HOGAR,FIBEX ARAGUA,PALO NEGRO,ATENCIÓN AL CLIENTE,Julio
2,A8251,501059438,INVERSIONES COLSANVE 21 CA,ACTIVO,2025-07-02,08:42:00,INFORMACION SERVICIO,FACTURACION,NANCY KARELIS DUARTE RIVAS OFIC METROPOLIS BQTO,40.0,PYMES,FIBEX ARAGUA,MARACAY,ATENCIÓN AL CLIENTE,Julio
3,A8251,501059438,INVERSIONES COLSANVE 21 CA,ACTIVO,2025-07-02,08:43:00,INFORMACION SERVICIO,FACTURACION,NANCY KARELIS DUARTE RIVAS OFIC METROPOLIS BQTO,40.0,PYMES,FIBEX ARAGUA,MARACAY,ATENCIÓN AL CLIENTE,Julio
4,B0602,14814547,AURIMAR OVIEDO VELA,ACTIVO,2025-07-03,10:02:00,RETIRO DE SERVICIO,MAL SERVICIO,LILIANA AGUILAR ADMIN OFIC PALMA DE ORO BARINAS,25.0,HOGAR,FIBEX BARINAS,BARINAS,ATENCIÓN AL CLIENTE,Julio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68216,V99348,7602644,LINDAY MARGARITA FONSECA GONZALEZ,ACTIVO,2026-01-09,10:26:00,RECLAMO DEL SERVICIO,COBRANZA,SORELYS HERRERA OFIC ALIANZA MALL,30.0,HOGAR,FIBEX VALENCIA,VALENCIA,ATENCIÓN AL CLIENTE,Enero
68217,VC2879,8822354,FROILASA MARIA MAURE REBOLLEDO,ACTIVO,2026-01-09,10:42:00,RECLAMO DEL SERVICIO,COBRANZA,REYMAR ELIZABET PARAO RIVAS OFIC VILLA DE CURA,30.0,HOGAR,FIBEX VILLA DE CURA,VILLA DE CURA,ATENCIÓN AL CLIENTE,Enero
68218,VC3116,11687818,JOSE NEPTALIS APARICIO,ACTIVO,2026-01-11,08:15:00,RECLAMO DEL SERVICIO,COBRANZA,REYMAR ELIZABET PARAO RIVAS OFIC VILLA DE CURA,30.0,HOGAR,FIBEX VILLA DE CURA,VILLA DE CURA,ATENCIÓN AL CLIENTE,Enero
68219,YG0759,17680271,DAYANA DAYANITH BARRIOS BARRIOS,RETIRADO,2026-01-09,15:08:00,RETIRO DE SERVICIO,CONTRATÓ OTRO PROVEEDOR,AGENTE AUTORIZADO SOLUCIONES TECNOLOGICA YORSI...,40.0,HOGAR,FIBEX YAGUA,YAGUA,ATENCIÓN AL CLIENTE,Enero
